<a href="https://cocl.us/Data_Science_with_Scalla_top"><img src = "https://s3-api.us-geo.objectstorage.softlayer.net/cf-courses-data/CognitiveClass/SC0103EN/adds/Data_Science_with_Scalla_notebook_top.png" width = 750, align = "center"></a>
 <br/>
<a><img src="https://ibm.box.com/shared/static/ugcqz6ohbvff804xp84y4kqnvvk3bq1g.png" width="200" align="center"></a>"

# Module 5: Pipeline and Grid Search

## Predicting Grant Applications: Building a Pipeline

### Lesson Objectives

* After completing this lesson, you should be able to extract useful information from the results of the grid search, including:
  - the average area under the ROC curve for each combination of parameters
  - the parameters of the best model
  - the feature importances of the best model
  
### avgMetrics

In [1]:
import $ivy.`org.apache.spark::spark-sql:2.4.0` // Or use any other 2.x version here
import $ivy.`org.apache.spark::spark-mllib:2.4.0` // Or use any other 2.x version here
import  org.apache.spark.SparkContext
import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)
val sc= new SparkContext("local[*]","Grant Applications")

Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties


import $ivy.$                                   // Or use any other 2.x version here

import $ivy.$                                     // Or use any other 2.x version here

import  org.apache.spark.SparkContext

import org.apache.log4j.{Level, Logger}

sc: SparkContext = org.apache.spark.SparkContext@441f1b7e

In [2]:
import org.apache.spark.sql.SparkSession
val spark = SparkSession.builder().getOrCreate()
import spark.implicits._
import org.apache.spark.sql.functions._

val data = spark.read.
  format("com.databricks.spark.csv").
  option("delimiter", "\t").
  option("header", "true").
  option("inferSchema", "true").
  load("data/grantsPeople.csv")

data.show()

+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+----+----+------------+------------+--------------------+-------------------+
|Grant_Application_ID| RFCD_Code|RFCD_Percentage| SEO_Code|SEO_Percentage|Person_ID|                Role|Year_of_Birth|Country_of_Birth|Home_Language| Dept_No|Faculty_No|With_PHD|No_of_Years_in_Uni_at_Time_of_Grant|Number_of_Successful_Grant|Number_of_Unsuccessful_Grant|  A2|   A|   B|   C|Grant_Status|Sponsor_Code| Contract_Value_Band|Grant_Category_Code|
+--------------------+----------+---------------+---------+--------------+---------+--------------------+-------------+----------------+-------------+--------+----------+--------+-----------------------------------+--------------------------+----------------------------+----+----+-

import org.apache.spark.sql.SparkSession

spark: SparkSession = org.apache.spark.sql.SparkSession@75a2884a
import spark.implicits._

import org.apache.spark.sql.functions._


data: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 22 more fields]

In [3]:

val researchers = data.
  withColumn ("phd", data("With_PHD").equalTo("Yes").cast("Int")).
  withColumn ("CI", data("Role").equalTo("CHIEF_INVESTIGATOR").cast("Int")).
  withColumn("paperscore", data("A2") * 4 + data("A") * 3)

val grants = researchers.groupBy("Grant_Application_ID").agg(
  max("Grant_Status").as("Grant_Status"),
  max("Grant_Category_Code").as("Category_Code"),
  max("Contract_Value_Band").as("Value_Band"),
  sum("phd").as("PHDs"),
  when(max(expr("paperscore * CI")).isNull, 0).
    otherwise(max(expr("paperscore * CI"))).as("paperscore"),
  count("*").as("teamsize"),
  when(sum("Number_of_Successful_Grant").isNull, 0).
    otherwise(sum("Number_of_Successful_Grant")).as("successes"),
  when(sum("Number_of_Unsuccessful_Grant").isNull, 0).
    otherwise(sum("Number_of_Unsuccessful_Grant")).as("failures")
)

grants.show()

+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|Grant_Application_ID|Grant_Status|Category_Code|          Value_Band|PHDs|paperscore|teamsize|successes|failures|
+--------------------+------------+-------------+--------------------+----+----------+--------+---------+--------+
|                 148|           0|  GrantCat30B|ContractValueBandUnk|null|         6|       1|        0|       1|
|                 463|           1|  GrantCat30C|ContractValueBandUnk|null|         0|       1|        1|       0|
|                 471|           0|  GrantCat30B|  ContractValueBandA|   1|       127|       2|        1|       5|
|                 496|           0|  GrantCat30B|  ContractValueBandA|null|         0|       1|        1|       3|
|                 833|           1|  GrantCat10A|  ContractValueBandF|null|         0|       1|        0|       0|
|                1088|           1|  GrantCat50A|  ContractValueBandA|   1|     

researchers: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, RFCD_Code: string ... 25 more fields]
grants: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]

In [4]:

import org.apache.spark.ml.feature.StringIndexer

val value_band_indexer = new StringIndexer().
  setInputCol("Value_Band").
  setOutputCol("Value_index").
  fit(grants)
  
val category_indexer = new StringIndexer().
  setInputCol("Category_Code").
  setOutputCol("Category_index").
  fit(grants)
  
val label_indexer = new StringIndexer().
  setInputCol("Grant_Status").
  setOutputCol("status").
  fit(grants)



import org.apache.spark.ml.feature.StringIndexer


value_band_indexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_90482f2eafdb
category_indexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_6594bf66333d
label_indexer: org.apache.spark.ml.feature.StringIndexerModel = strIdx_0ab1e0e8dff5

In [5]:
import org.apache.spark.ml.feature.VectorAssembler

val assembler = new VectorAssembler().
  setInputCols(Array(
    "Value_index"
    ,"Category_index"
    ,"PHDs"
    ,"paperscore"
    ,"teamsize"
    ,"successes"
    ,"failures"
  )).setOutputCol("assembled")


import org.apache.spark.ml.feature.VectorAssembler


assembler: VectorAssembler = vecAssembler_954371b83f3a

In [6]:
import org.apache.spark.ml.classification.RandomForestClassifier
import org.apache.spark.ml.classification.RandomForestClassificationModel

val rf = new RandomForestClassifier().
  setFeaturesCol("assembled").
  setLabelCol("status").
  setSeed(42)

import org.apache.spark.ml.Pipeline
val pipeline = new Pipeline().setStages(Array(
    value_band_indexer,
    category_indexer,
    label_indexer,
    assembler,
    rf)
  )

import org.apache.spark.ml.classification.RandomForestClassifier

import org.apache.spark.ml.classification.RandomForestClassificationModel


rf: RandomForestClassifier = rfc_d1d806280ae9
import org.apache.spark.ml.Pipeline

pipeline: Pipeline = pipeline_cbd6a2aacd4e

In [7]:
import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator
val auc_eval = new BinaryClassificationEvaluator().
  setLabelCol("status").
  setRawPredictionCol("rawPrediction")

auc_eval.getMetricName

val tr = grants.filter("Grant_Application_ID < 6635")
val te = grants.filter("Grant_Application_ID >= 6635")
val training = tr.na.fill(0, Seq("PHDs"))
val test = te.na.fill(0, Seq("PHDs"))

val model = pipeline.fit(training)
val pipeline_results = model.transform(test)
auc_eval.evaluate(pipeline_results)

rf.extractParamMap


import org.apache.spark.ml.evaluation.BinaryClassificationEvaluator

auc_eval: BinaryClassificationEvaluator = binEval_c36a3e1f931f
res6_2: String = "areaUnderROC"
tr: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
te: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
training: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
test: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 7 more fields]
model: org.apache.spark.ml.PipelineModel = pipeline_cbd6a2aacd4e
pipeline_results: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 14 more fields]
res6_9: Double = 0.9107593972299525
res6_10: org.apache.spark.ml.param.ParamMap = {
	rfc_d1d806280ae9-cacheNodeIds: false,
	rfc_d1d806280ae9-checkpointInterval: 10,
	

In [8]:
import org.apache.spark.ml.tuning.ParamGridBuilder

val paramGrid = new ParamGridBuilder().
  addGrid(rf.maxDepth, Array(2, 5)).
  addGrid(rf.numTrees, Array(1, 20)).
  build()

import org.apache.spark.ml.tuning.ParamGridBuilder


paramGrid: Array[org.apache.spark.ml.param.ParamMap] = Array(
  {
	rfc_d1d806280ae9-maxDepth: 2,
	rfc_d1d806280ae9-numTrees: 1
},
  {
	rfc_d1d806280ae9-maxDepth: 2,
	rfc_d1d806280ae9-numTrees: 20
},
  {
	rfc_d1d806280ae9-maxDepth: 5,
	rfc_d1d806280ae9-numTrees: 1
},
  {
	rfc_d1d806280ae9-maxDepth: 5,
	rfc_d1d806280ae9-numTrees: 20
}
)

In [9]:
import org.apache.spark.ml.tuning.CrossValidator

val cv = new CrossValidator().
  setEstimator(pipeline).
  setEvaluator(auc_eval).
  setEstimatorParamMaps(paramGrid).
  setNumFolds(3)

val cvModel = cv.fit(training)

val cv_results = cvModel.transform(test)
cvModel.avgMetrics

import org.apache.spark.ml.tuning.CrossValidator


cv: CrossValidator = cv_d0f8e2d95a0b
cvModel: org.apache.spark.ml.tuning.CrossValidatorModel = cv_d0f8e2d95a0b
cv_results: org.apache.spark.sql.package.DataFrame = [Grant_Application_ID: int, Grant_Status: int ... 14 more fields]
res8_4: Array[Double] = Array(
  0.6957094062620838,
  0.8640417833958612,
  0.9069637783874809,
  0.9220716323150743
)

### Finding the Winning Parameters

In [10]:
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.tuning.CrossValidatorModel

implicit class BestParamMapCrossValidatorModel(cvModel: CrossValidatorModel)
{
  def bestEstimatorParamMap: ParamMap = cvModel.getEstimatorParamMaps.zip(cvModel.avgMetrics).maxBy(_._2)._1
}

import org.apache.spark.ml.param.ParamMap

import org.apache.spark.ml.tuning.CrossValidatorModel


defined class BestParamMapCrossValidatorModel

### Using bestEstimatorParamMap

In [11]:

println(cvModel.bestEstimatorParamMap)


{
	rfc_d1d806280ae9-maxDepth: 5,
	rfc_d1d806280ae9-numTrees: 20
}


### Best Model

In [12]:
val bestPipelineModel = cvModel.bestModel.asInstanceOf[org.apache.spark.ml.PipelineModel]
bestPipelineModel.stages

bestPipelineModel: org.apache.spark.ml.PipelineModel = pipeline_cbd6a2aacd4e
res11_1: Array[org.apache.spark.ml.Transformer] = Array(
  strIdx_90482f2eafdb,
  strIdx_6594bf66333d,
  strIdx_0ab1e0e8dff5,
  vecAssembler_954371b83f3a,
  RandomForestClassificationModel (uid=rfc_d1d806280ae9) with 20 trees
)

### Extracting the Winning Classifier

In [13]:
val bestRandomForest = bestPipelineModel.stages(4).asInstanceOf[RandomForestClassificationModel]
bestRandomForest.toDebugString

bestRandomForest: RandomForestClassificationModel = RandomForestClassificationModel (uid=rfc_d1d806280ae9) with 20 trees
res12_1: String = """RandomForestClassificationModel (uid=rfc_d1d806280ae9) with 20 trees
  Tree 0 (weight 1.0):
    If (feature 6 <= 0.5)
     If (feature 2 <= 0.5)
      If (feature 5 <= 0.5)
       If (feature 1 in {0.0,1.0,2.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0})
        If (feature 3 <= 6.5)
         Predict: 0.0
        Else (feature 3 > 6.5)
         Predict: 1.0
       Else (feature 1 not in {0.0,1.0,2.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0})
        Predict: 1.0
      Else (feature 5 > 0.5)
       If (feature 0 in {2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0})
        Predict: 1.0
       Else (feature 0 not in {2.0,3.0,4.0,5.0,6.0,7.0,8.0,10.0})
        If (feature 5 <= 4.5)
         Predict: 1.0
        Else (feature 5 > 4.5)
         Predict: 0.0
     Else (feature 2 > 0.5)
      If (feature 1 in {1.0,4.0,5.0,6.0,7.0,8.0,9.0,11.0,12.0})
       If (feature 5 <= 0.5)
        If (fea

### totalNumNodes

In [14]:
bestRandomForest.totalNumNodes

res13: Int = 756

### Feature Importances

In [15]:
bestRandomForest.featureImportances

res14: org.apache.spark.ml.linalg.Vector = (7,[0,1,2,3,4,5,6],[0.19192957644290337,0.18113930716037552,0.011889584580838638,0.034636050385032285,0.004203668870944973,0.21741367469495718,0.3587881378649479])

### Wrapping Up

* Using the default parameters, we had an area under the ROC curve of 0.909
* After a grid search, we got that up to 0.926
* Running the grid search on a cluster was a real timesaver
* Not all of our features proved very useful; maybe you can do better!

### Module Summary

* Having completed this module about Predicting Grant Applications, you should be able to:
  - Understand how to fit together the functions available in Spark's machine learning libraries to solve real problems
  - Fit models in a fraction of the time, using a Spark cluster


### About the Authors

[Petro Verkhogliad](https://www.linkedin.com/in/vpetro) is Consulting Manager at Lightbend. He holds a Masters degree in Computer Science with specialization in Intelligent Systems. He is passionate about functional programming and applications of AI.

In [16]:
sc.stop()